In [7]:
import numpy as np
import torch.nn as nn
import torch
import torch.optim as optim
from torchvision import datasets, transforms
import emnist
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import loader
mnist_images, mnist_labels = emnist.extract_test_samples('digits')

train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=True)

# animals_dataset = loader.load_chapel_dataset('./animals-10')

In [8]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1,out_channels=8,kernel_size=7),
    nn.Conv2d(in_channels=8,out_channels=12,kernel_size=5),
    nn.MaxPool2d(kernel_size=2),
    nn.Softmax(dim=1)
)

In [9]:
model

Sequential(
  (0): Conv2d(1, 8, kernel_size=(7, 7), stride=(1, 1))
  (1): Conv2d(8, 12, kernel_size=(5, 5), stride=(1, 1))
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Softmax(dim=1)
)

In [10]:
img = mnist_images[0,:,:]
img = torch.from_numpy(img.reshape(1,1,28,28)).float()
img.shape

torch.Size([1, 1, 28, 28])

In [11]:
model(img).shape
model(img)

tensor([[[[1.0000e+00, 5.6172e-07, 2.7871e-11, 4.8789e-19, 5.5263e-37,
           1.3277e-39, 2.3858e-21, 0.0000e+00, 0.0000e+00],
          [1.5352e-02, 2.2421e-19, 2.6616e-20, 5.9123e-36, 9.1094e-37,
           5.8457e-23, 5.8973e-07, 1.7175e-36, 8.5150e-36],
          [1.1132e-22, 1.6347e-36, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           1.1009e-35, 4.5940e-10, 2.9219e-37, 4.1396e-24],
          [2.4943e-43, 8.6600e-43, 0.0000e+00, 0.0000e+00, 1.8749e-42,
           7.1464e-28, 6.3926e-15, 1.6894e-37, 6.4993e-28],
          [5.3833e-36, 0.0000e+00, 0.0000e+00, 9.8264e-24, 1.5825e-02,
           1.0000e+00, 8.4265e-22, 3.0447e-32, 0.0000e+00],
          [1.9651e-38, 0.0000e+00, 7.8240e-39, 1.6150e-03, 9.9877e-01,
           1.5356e-12, 1.0189e-29, 0.0000e+00, 0.0000e+00],
          [2.3377e-25, 4.9911e-26, 1.4096e-38, 9.9995e-01, 5.1198e-07,
           1.4139e-23, 2.3993e-35, 0.0000e+00, 4.6220e-35],
          [1.0000e+00, 9.9894e-01, 3.8829e-05, 2.0819e-13, 2.6606e-37,
           5

In [12]:


epochs = 40
learning_rate = 0.05
batch_size = 10
num_train_images = 20000

loss_criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

model.train()

for epoch in range(epochs):
    epoch_loss = 0
    num_pairs = len(train_dataset)# mnist_images.shape[0]
    num_batches = num_train_images // batch_size
    with tqdm(total=num_batches) as pbar:
        bc = 0
        for batch, labels in train_loader:
            if bc > num_batches:
                break
            bc += 1
            # for b in range(num_batches):
            # batch = mnist_images[b*batch_size:(b+1)*batch_size,:,:]
            # batch = torch.from_numpy(batch.reshape(batch_size,1,28,28)).float() / 255
            # labels = torch.from_numpy(mnist_labels[b*batch_size:(b+1)*batch_size])

            optimizer.zero_grad()
            output = model(batch)
            loss = loss_criterion(output,labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            pbar.update(1)
            # print(loss.item())
        # for (im_idx, label) in zip(range(num_pairs), mnist_labels):
        #     im = mnist_images[im_idx,:,:]
        #     im = torch.from_numpy(im.reshape(1,1,28,28)).float()

        #     optimizer.zero_grad()
        #     output = model(im)
        #     label_vector = torch.zeros(1,10)
        #     label_vector[0,label] = 1
        #     loss = loss_fn(output,label_vector)# -torch.sum(torch.log(output))
        #     loss.backward()
        #     optimizer.step()
        #     # print(loss.item(),output[0,label].item())
        #     epoch_loss += loss.item() 
        #     pbar.update(1)
    print('loss: ', epoch_loss / num_pairs)
    

/var/folders/wt/nr12w9r958d0nwkk86zwr9600000gp/T/ipykernel_18432/3700079496.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm(total=num_batches) as pbar:


  0%|          | 0/2000 [00:00<?, ?it/s]

RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

model.eval()
correct = 0
for data,target in test_loader:
    output = model(data)
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
print('accuracy: ', correct ,'/', len(test_dataset))

In [ ]:
(im,lb) = next(iter(train_loader))
lb.shape

In [ ]:
transforms.ToTensor()